In [1]:
import sys
from google.colab import drive

drive.mount('/content/drive', force_remount=True)
sys.path.append('/content/drive/MyDrive/finance_course/2021/lesson6')

Mounted at /content/drive


## Value at Risk

* The Value at Risk (VaR) of a portfolio is used to know to a certain confidence level how much will be the maximum loss in the next $N$ days. 
  * It is a function of two parameters: the time horizon (i.e. N days) and the confidence level. 
  * It can be interpreted as the loss level over a certain time horizon that has a probability of only (100 − X)% of being exceeded.
  * Usually N is 1 day and X is 95.
* The VaR is the **loss corresponding to the (100 − X)th percentile of the portfolio change in value distribution over the next N days**.

![](https://drive.google.com/uc?id=1TNNlBn_Xl-F5BLYxJR7uEDSdzouGOafU)

* VaR is useful to summarize all the information about the risk of a portfolio in one single number
  * this can also be considered its main limitation as it implies too much simplification of such a complex task.

## How to Estimate VaR
* In the following historical series of Apple and Netflix are used ([historical_data.csv](https://raw.githubusercontent.com/matteosan1/finance_course/develop/libro/input_files/historical_data.csv)):
  * it has been assumed a portfolio made of 60% of AAPL and 40% NFLX.

In [3]:
# read historical series and define weights
import pandas as pd
import numpy as np

df = pd.read_csv("https://raw.githubusercontent.com/matteosan1/finance_course/develop/libro/input_files/historical_data.csv", index_col="Date")
print (df.head())

w = np.array([0.6, 0.4])

                aapl      nflx
Date                          
2010-01-04  6.553025  7.640000
2010-01-05  6.564356  7.358571
2010-01-06  6.459940  7.617143
2010-01-07  6.447998  7.485714
2010-01-08  6.490867  7.614286


* In the following we are going to add a new column with the daily return to the dataframe.
  * **Arithmetic return**:
$$r_{arithm} = \frac{FV}{PV} - 1 = \frac{FV - PV}{PV}\quad(\texttt{.pct_change()})$$
    * not symmetric: if a position appreciates 15% and then depreciates 15%, the total change is -2.25%.
$$FV = PV(1 + r_{arithm})\\[5pt]$$
$$FV = PV(1+.15)(1 - .15) = PV(0.9775)\\[5pt]$$
$$\frac{FV}{PV} - 1 = 0.9775 - 1 = -2.25 \%$$
  * **Logaritmic return**:
$$r_{log} = \log\left(\frac{FV}{PV}\right)$$
    * symmetric:
$$FV = PV e^{r_{log}}\\[5pt]$$
$$FV = PV e^{0.1398} e^{-0.1398} = PV\\[10pt]$$

$$r_{arithm} + 1 = \frac{FV}{PV} \implies \log(r_{arithm} + 1) = \log\left(\frac{FV}{PV}\right) \implies r_{log} = \log(1 + r_{arithm})$$ 

* For small variations $r_{arithm}$ and $r_{log}$ are very similar.

In [5]:
# add daily log-returns

df['aapl_rets'] = np.log1p(df['aapl'].pct_change())
df['nflx_rets'] = np.log1p(df['nflx'].pct_change())
print (df.head())

                aapl      nflx  aapl_rets  nflx_rets
Date                                                
2010-01-04  6.553025  7.640000        NaN        NaN
2010-01-05  6.564356  7.358571   0.001728  -0.037532
2010-01-06  6.459940  7.617143  -0.016034   0.034536
2010-01-07  6.447998  7.485714  -0.001850  -0.017405
2010-01-08  6.490867  7.614286   0.006626   0.017030


* Portfolio value $\Pi$ is determined by the *scalar product* between the invested amount (the weights $w_i$) and the asset values ($v_i$), i.e. $\Pi = \sum_i w_i \cdot v_i$. 
  * In $\texttt{python}$, when using $\texttt{numpy.array}$ for weights and values, it is indicated with the method $\texttt{.dot}$. 
  * For some practical calculations it may be useful to multiply two vectors compents by components without summing up at the end all the results and this is done with the operator $\texttt{*}$.

$$
\begin{bmatrix}
1 & 2 \\
3 & 4 \\
\end{bmatrix}*[10, 20] = 
\begin{bmatrix}
1*10 & 2*20 \\
3*10 & 4*20 \\
\end{bmatrix} = 
\begin{bmatrix}
10 & 40 \\
30 & 80 \\
\end{bmatrix}\\[10pt]
$$

$$[10, 20]\cdot[5, 6] = 10*5 + 20*6 = 170$$

In [6]:
import numpy as np
a = np.array([[1, 2], [3, 4]])
b = np.array([10, 20])
c = np.array([5, 6])

print (a*b)
print (b.dot(c))

[[10 40]
 [30 80]]
170


* **To determine VaR there are essentially two methods: historical or Monte Carlo simulation.**

### Historical Simulation

* The historical sequence of the asset daily price variations will provide different scenarios to be applied to today’s asset values. 

* The daily evolution of the portfolio can be simulated by rescaling each asset value according to its variation between two consecutive days $i$ and $i − 1$

$$\Delta\Pi = \left(\cfrac{v_1(t_i)}{v_1(t_{i−1})} - 1\right)v_1(t_n)w_1 + \left(\cfrac{v_2(t_i)}{v_2(t_{i−1})} - 1\right)v_2(t_n)w_2$$

* Draw the portfolio variation distribution, and the VaR estimate will be its (100-X)-percentile.
  * Such estimates relies on the assumption that past behaviors are indicative of what might happen in the future, 
  * it is important that historical series was as large as possible.


In [11]:
# historical VaR with for-loops

dP = []
for i in range(1, len(df)):
  dP.append(df['aapl_rets'].iloc[i]*df['aapl'].iloc[-1]*w[0] + df['nflx_rets'].iloc[i]*df['nflx'].iloc[-1]*w[1] )

hist_var = np.percentile(dP, 5)
print (hist_var)


[-9.691872280318176, 7.611123825392187, -4.734440307663996, 5.068154251554154, -1.1466039305296731, -5.309161647545682, 9.115464719982832, -15.380712720059044, -1.7309396206553387, 5.20133847463753, -1.1987290083914943, -1.8340061403677985, -6.145434428864806, -6.833290036813935, 5.613249751505117, 6.152663333197108, 51.949111442080735, -6.652823774823239, -3.9443533878715833, 4.025689876693808, 1.967579224117759, -10.230739034596265, 3.757914575582711, -0.3638343462887958, 6.206629567497653, -3.211838690851359, 9.492419898534276, 0.028981045124906735, 8.239692183887035, -1.7986801485716244, 6.55852094878912, 1.6107405707705484, -6.0525675525196965, -5.254740121820701, 3.0209110700556647, 4.307651483797526, 2.9199103536634525, 16.025974715517147, -2.785299239126243, -5.2890865373894265, 2.375376373570125, 2.078439159630189, 4.018731262582165, 6.383149083274187, 5.715183581425586, -0.06630019033997875, -4.057888327837723, -2.5376496948983993, 4.100984161942183, 0.9420184312669974, -1.19

<img src="https://drive.google.com/uc?id=15Tkz8_iOy7NVft_8Ia6vDpP4ShuSAqlb">

In [13]:
from scipy.stats import norm, t

t_params = t.fit(dP)
g_params = norm.fit(dP)

print (norm(*g_params).ppf(0.05))
print (t(*t_params).ppf(0.05))


-14.089512178475784
-12.05687425075882


* By fitting the distribution of the changes in portfolio value it is apparent how it is a t-student rather than a Gaussian.

### Monte Carlo Simulation
* A very useful alternative to the historical approach is to use Monte Carlo simulation to generate the distribution of $\Delta\Pi$.

* The simulation can be done in two alternative ways:
  * generating random returns from a distribution with mean and standard deviation obtained from the historical data of each asset;
  * evolving each asset price using a geometric Brownian motion.

#### First Option
* Compute mean and standard deviation from the historical dataset, then sample various simulated returns from a multivariate Gaussian with such mean and variance. 
  * One useful aspect of this method is that other distributions than Gaussian could be used.


In [17]:
# MC VaR with distribution
from scipy.stats import multivariate_normal

mean = df[['aapl_rets', 'nflx_rets']].mean()
print (mean)
cov = df[['aapl_rets', 'nflx_rets']].cov()
print (cov)

mv = multivariate_normal(mean=mean, cov=cov)
x = mv.rvs(size=10000)

dP = []
for i in range(10000):
  dP.append(x[i,0]*df['aapl'].iloc[-1]*w[0] + x[i,1]*df['nflx'].iloc[-1]*w[1] )

mc_var = np.percentile(dP, 5)
print (mc_var)

aapl_rets    0.001051
nflx_rets    0.001492
dtype: float64
           aapl_rets  nflx_rets
aapl_rets   0.000313    0.00015
nflx_rets   0.000150    0.00100
-14.055666183235646


<img src="https://drive.google.com/uc?id=1TGgzTD1czT0YJUV-BgDyc1eZ7OE39WHv">

* This result can be compared to the VaR estimate determined by evolving the asset price. 
  * We will use the geometric Brownian motion where $\mu$ and $\sigma$ are the mean and variance estimated from the historical series. 

In [18]:
# MC VaR with evolution
from numpy.random import normal, seed

T=1
dP = []
current_price = df['aapl'].iloc[-1]*w[0]+df['nflx'].iloc[-1]*w[1]
for _ in range(10000):
  P0 = df['aapl'].iloc[-1]*np.exp((mean[0]-0.5*cov.iloc[0,0])*T + np.sqrt(cov.iloc[0,0]*T)*normal())
  P1 = df['nflx'].iloc[-1]*np.exp((mean[1]-0.5*cov.iloc[1,1])*T + np.sqrt(cov.iloc[1,1]*T)*normal())

  P = P0*w[0] + P1*w[1]
  dP.append(P-current_price)

mc_var2 = np.percentile(dP, 5)
print (mc_var2)


-13.055316172489244


<img src="https://drive.google.com/uc?id=1Dd_Z2lAgfR9TRA0C_xaELLt4zDXMHOpw">

### Stress-test and Backtesting

* It is generally useful to check how VaR would behave under the most extreme market moves seen in the last years.
  * This kind of test is called **stress-test**;
  * it is done by extracting from the historical series particular days with exceptionally large variation of the market variables, to take into account extreme events that can happen more frequently in reality than in a simulation (where usually Gaussian tails are assumed). 
  * For example a 5-standard deviation move is expected to happen once every 7000 years but in practice can be observed twice over 10 years.

* **Backtesting** consists of assessing how well the VaR estimate would have performed in the past. 
  * Basically it has to be tested how often the daily loss exceeded the daily X% VaR just computed. 
    * If it happens on about (100- X)% of the times we can be confident that our estimate is correct. 
    * Clearly back-testing makes sense only if VaR has been estimated on an independent historical sample with respect to that used in the test.

## Credit VaR

* Credit VaR is defined as a percentile of the **credit loss** distribution. 
  * In this case we are concerned with the default risk associated to counter-parties instead of the market risk.
* The **exposure** $EE(\tau)$ defined as the sum of the discounted cash flows at the default date $\tau$. 
* The corresponding **loss** is then given by $L =(1−R)·EE(\tau)$
  * where $L$ is non-zero only in scenarios of early counter-party default.
* Credit VaR can be expressed as the X-quantile of the distribution of $L$. 
  * Time horizon is usually set to one year and the percentile to the 99.9th, so it returns the loss that is exceeded only in 1 case out of 1000.

### Credit VaR and MC Simulation
* Credit VaR can be calculated through a simulation of the evolution of a portfolio up to the risk horizon, including possible defaults of the counter-parties.
* In each experiment the portfolio is priced obtaining a number of scenarios to draw the loss distribution.

#### Example
* Consider a portfolio of 20 zero coupon bonds each one with a default probability of 8% and the same face value (100 EUR). The recovery rate in case of default is 40% and the risk free rate is 1%.

In [19]:
# compute credit var of 20 ZCB
import numpy as np
from datetime import date
from scipy.stats import uniform

bonds = 20
DP = 0.08
FV = 100
R = 0.4
r = 0.01
df = np.exp(-r)
L = []
for _ in range(10000):
  u = uniform.rvs(size=bonds)
  n_defaults = (u<DP).sum()
  L.append((1-R)*FV*df*n_defaults)

print (np.percentile(L, 99.9))

356.4179401497005


In [21]:
from scipy.stats import binom

b = binom(bonds, DP)
print (b.ppf(0.999)*(1-R)*FV*df)

356.4179401497005


* Actually, since we are dealing with independent and equiprobable default events, the distribution of the losses could have been estimated simply with the **binomial distribution**.
  * Loss distribution is a scaled version of the distribution of defaults, since LGD and FV are the same for each ZCB.


<table>
<tr>
<td><img src="https://drive.google.com/uc?id=1fstX-PUqjhFr1Xujoeam9yxw3_01jhOS"></td>
<td><img src="https://drive.google.com/uc?id=1L0P2889QJJSGfqq3zXtzCO5LXnPgaE_Y"></td>
</tr>
</table>

## CVA and DVA

* Suppose you have a portfolio of derivatives. 
* If a counter-party defaults and the present value of the portfolio at default is positive to the surviving party (you), then the actual gain is only given by the recovery fraction of the value. 
* If however the present value is negative to you, you have to pay it in full to the liquidators of the defaulted entity.
* This behaviour creates an asymmetry which can be corrected by changing the definition of the deal value as the value without counter-party risk minus a positive adjustment, called Credit Valuation Adjustment (CVA).

$$\textrm{CVA} = (1-R)\int_0^{T} D(t)\cdot EE(t) dQ(t)$$

* where $T$ is the latest maturity in the portfolio, $D$ is the discount factor, $EE$ is the expected exposure or $\mathbb{E}[\textrm{max}(0, \mathrm{NPV_{portfolio}})]$, and $dQ$ is the probability of default between $t$ and $t + dt$.

* Or it's discrete version:

$$\textrm{CVA} = (1-R)\sum_i D(t_i)\cdot EE(t_i)\cdot  Q(t_{i-1}, t_i))$$

* **Credit VaR measures the risk of losses faced due to the default of some counter-party, while CVA measures the price adjustment of a contract due to this risk**.

### DVA
* The adjustment seen from the point of view of our counter-party is positive, and is called Debit Valuation Adjustment, DVA. 
* It is positive because the early default of the client itself would imply a discount on its payment obligations, and this means a gain. 

* When both parties have a non-null probability of default, they consistently include both CVA and DVA into the valuation. So they will mark **a positive CVA to be subtracted** and **a positive DVA to be added** to the default-risk-free price of the deal. 
  * The CVA of one party will be the DVA of the other one and vice versa.

$$\textrm{price = default risk free price + DVA - CVA}$$

## CVA Computation

* CVA can be computed with Monte Carlo simulation. 
  1. Compute the portfolio value at each time point for each MC scenario.    
  2. Calculate the CVA using one of the equation above. 
  3. Average the CVA of all the scenarios to get its estimate.

#### Example
* Imagine a 3-years zero coupon bond with a face value FV = 100. 
* The bond issuer has the following default probabilities 10%, 20% and 30% for 1, 2 and 3 years respectively and the recovery rate is 40%. 
* The risk free rate is instead 3% flat.


In [25]:
# CVA of a ZCB
from dateutil.relativedelta import relativedelta
from finmarkets import DiscountCurve, CreditCurve
import math

FV = 100
T = 3
r = 0.03
R = 0.4
pricing_date = date.today()
pillar_dates = [pricing_date + relativedelta(years=i) for i in range(T+1)]
dfs = [np.exp(-r*t) for t in range(T+1)]
dc = DiscountCurve(pillar_dates, dfs)
S = [1, 0.9, 0.8, 0.7]
cc = CreditCurve(pillar_dates, S)

cva = 0 
d = pricing_date
while d <= pillar_dates[-1]:
  cva += (1-R)*dc.df(d)*max(0, FV)*(cc.ndp(d)-cc.ndp(d+relativedelta(days=1)))
  d += relativedelta(days=1)

print (cva)
print (FV*np.exp(-r*3)-cva)

17.21446974529636
74.17864878182647


* This is a simplified situation in which one scenario is enough: 
  * no simulation of ZCB paramaters is needed (eg. interest rate).